# Lab 2.1: LangGraph Persistence

In this lab, we will add **Persistence** (Memory) to our graph. This allows the graph to remember the state across different interactions, enabling long-running conversations.

## Key Concepts
1. **Checkpointer**: A mechanism to save the state of the graph at every step.
2. **MemorySaver**: An in-memory checkpointer for development/testing.
3. **Thread ID**: A unique identifier to separate different user sessions.

In [ ]:
# 1. Install Dependencies
%pip install -qU langchain-groq langchain-community langgraph

In [1]:
# 2. Setup API Keys
import getpass
import os

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

In [3]:
# 3. Define Graph (Same as Lab 1.3)
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_groq import ChatGroq

class State(TypedDict):
    messages: Annotated[list, add_messages]
    
# Initialize LLM
llm = ChatGroq(
    model="qwen/qwen3-32b",
    temperature=0,
    reasoning_format="parsed"
)

def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}

graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

## 4. Add Persistence
We use `MemorySaver` to store the state.

In [4]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

# Compile with checkpointer
graph = graph_builder.compile(checkpointer=memory)

## 5. run with Thread ID
We use `configurable` to specify the thread.

In [5]:
config = {"configurable": {"thread_id": "1"}}

user_input = "Hi, I am Bob."

for event in graph.stream({"messages": [("user", user_input)]}, config=config):
    for value in event.values():
        print("Assistant:", value["messages"][-1].content)

Assistant: Hi Bob! Nice to meet you. How can I assist you today? 😊


## 6. Verify Memory
Ask "What is my name?" in a new execution block using the SAME `thread_id`.

In [6]:
user_input = "What is my name?"

# Note: We are using the same 'config' with thread_id='1'
for event in graph.stream({"messages": [("user", user_input)]}, config=config):
    for value in event.values():
        print("Assistant:", value["messages"][-1].content)

Assistant: Your name is Bob! 😊 You mentioned it earlier. How can I assist you today, Bob?


## 7. New Thread
If we change the `thread_id`, the memory should be empty.

In [7]:
config_new = {"configurable": {"thread_id": "2"}}

# Asking the same question in a new thread
user_input = "What is my name?"

for event in graph.stream({"messages": [("user", user_input)]}, config=config_new):
    for value in event.values():
        print("Assistant:", value["messages"][-1].content)

Assistant: I don't have access to your personal information, including your name. If you'd like me to address you by name in our conversation, feel free to share it with me!
